### 光流估计

#### 定义
1. 光流是视频中**相邻帧之间像素运动的矢量描述**，反应物体在图像平面上的瞬时运动场（瞬时速度）。根据各个像素点的速度矢量特征，可以对图像进行动态分析，例如目标跟踪。

2. 通过当前帧与下一帧像素运动变化，可以分析出像素点瞬时速度大小、方向等信息。

#### 核心假设（前提条件）
1. 亮度恒定：同一像素点在连续帧中的亮度不变
2. 时间连续：运动位移量较小（可通过金字塔缓解）
3. 空间一致：邻近像素有相似运动（领域约束）。一个场景上邻近的点投影到图像上也是邻近点，且邻近点速度一致。因为光流法基本方程约束只有一个，而要求x，y方向的速度，有两个未知变量。所以需要连立n多个方程求解。

#### 核心算法
【Lucas-Kanade】下边介绍

#### 应用场景
1. 自动驾驶：障碍物运动估计、碰撞预警
2. 视频稳定：相机运补偿
3. 行为识别：人体动作分析（如跌倒检测）
4. 医学影响：心脏运动分析、血流跟踪
5. 增强现实：虚拟物体与真实场景运动同步



![title](lk_2.png)

![title](lk_1.png)

### Lucas-Kanade 算法

#### 核心算法原理
1. 基础假设（和上述一致【亮度恒定、小运动、空间一致性】）
2. 数学推导：见下图
3. 金字塔分层（Pyramid）：为解决大位移问题，算法在图像金字塔（多层缩放图像）上运行。


![title](lk_3.png)

如何求解方程组呢？看起来一个像素点根本不够，在物体移动过程中还有哪些特性呢？

![title](lk_4.png)

### cv2.calcOpticalFlowPyrLK():
参数：
- prevImage：前一帧图像（单通道灰度图）

- nextImage：当前帧图像（单通道灰度图）

- prevPts：待跟踪的特征点向量（前一帧的特征点，一般都是图像的角点）

- winSize：搜索窗口的大小（默认(15.15)）

- maxLevel：最大的金字塔层数（金字塔层（0表示不使用金字塔））


返回：

- nextPts：当前帧中匹配的特征点坐标（无效点坐标为NaN）

- status：特征点是否找到，标志数组（1表示成功跟踪，0表示丢失）

- err: 每个特征点的误差

In [ ]:
import numpy as np
import cv2

cap = cv2.VideoCapture('test.avi')

# 角点检测所需参数
# maxCorners：检测的最大角点数量
# qualityLevel：角点质量阈值（0-1之间），值越小检测到的角点越多
# minDistance：角点之间的最小欧氏距离
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7)

# Lucas-Kanade光流算法参数
# winSize：每个金字塔级别的搜索窗口大小
# maxLevel：金字塔层数（0表示不使用金字塔）
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2)

# 随机颜色条
# 生成100个随机BGR颜色，用于可视化跟踪轨迹
color = np.random.randint(0,255,(100,3))

# 拿到第一帧图像
# ret表示是否成功读取，old_frame是图像数据
ret, old_frame = cap.read()
# 将第一帧图像转换为灰度图（光流算法通常在灰度图像上运行）
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
# 使用Shi-Tomasi角点检测第一帧特征点，返回检测到的特征点坐标
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

# 创建一个与第一帧大小相同的全黑mask（掩膜），用于绘制跟踪轨迹
mask = np.zeros_like(old_frame)

while(True):
    # 开始视频处理循环，读取下一帧
    ret,frame = cap.read()
    # 转换为灰度图
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 需要传入前一帧和当前图像以及前一帧检测到的角点
    # 使用Lucas-Kanade算法计算稀疏光流
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # st=1表示：筛选出成功跟踪的特征点(st=1的点)
    # good_new是当前帧中成功跟踪的点
    # good_old是前一帧中对应的点
    good_new = p1[st==1]
    good_old = p0[st==1]

    # 绘制轨迹
    # 遍历所有成功跟踪的点对
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        # 解包新旧点的坐标
        a,b = new.ravel()
        c,d = old.ravel()

        # 将坐标调整为int类型
        a, b = int(a), int(b)
        c, d = int(c), int(d)
        # 在mask图像上绘制从旧点到新点的线段（跟踪轨迹）
        # 使用预先分配的颜色，线宽为2像素
        mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        # 在当前帧图像上绘制新点位置（半径为5像素的实心圆）
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
    # 将当前帧和轨迹mask叠加在一起
    img = cv2.add(frame,mask)

    # 展示图像
    cv2.imshow('frame',img)
    k = cv2.waitKey(150) & 0xff
    if k == 27:
        break

    # 更新前一帧的灰度图像和角点位置，为下一帧处理做准备
    # 将good_new重新整形为与p0相同的形状（n×1×2）
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)

cv2.destroyAllWindows()
cap.release()